In [2]:
# parameters and packages
import re

match = 2
mismatch = 1
gap = 2

In [3]:
def GlobalAlignmentBacktrack(v, w, indel, match_reward, mismatch_penalty):
    vLen = len(v)
    wLen = len(w)
    backtrack = [[0] * (vLen + 1) for _ in range(wLen + 1)]
    s = [[0] * (vLen + 1) for _ in range(wLen + 1)]

    for i in range(1, wLen + 1):
        s[i][0] = s[i-1][0] - indel
        backtrack[i][0] = "down"
    for j in range(1, vLen + 1):
        s[0][j] = s[0][j-1] - indel
        backtrack[0][j] = "right"

    for i in range(1, wLen + 1):
        for j in range(1, vLen + 1):
            w_letter = w[i-1]
            v_letter = v[j-1]

            # determine if match or mismatch
            if w_letter == v_letter:
                diag = s[i-1][j-1] + match_reward
            else:
                diag = s[i-1][j-1] - mismatch_penalty

            right = s[i][j-1] - indel
            down = s[i-1][j] - indel

            s[i][j] = max(right, down, diag)

            if s[i][j] == down:
                backtrack[i][j] = "down"
            elif s[i][j] == right:
                backtrack[i][j] = "right"
            elif s[i][j] ==  diag:
                backtrack[i][j] = "diagonal"

    return backtrack, s

def OutputGlobalAlignment(Backtrack, str2, str1):
    align1 = ""
    align2 = ""
    i = len(str2)
    j = len(str1)

    while not(i == 0 and j == 0):
        if Backtrack[i][j] == "down": # insertion in string 1 
            i = i-1
            align2 = str2[i] + align2
            align1 = "-" + align1

        elif Backtrack[i][j] == "right": # deletion in string 1 
            j = j-1
            align2 = "-" + align2
            align1 = str1[j] + align1
        else:
            i = i-1
            j = j-1
            align2 = str2[i] + align2
            align1 = str1[j] + align1

    return align1, align2

# match_reward=match, mismatch_penalty=mismatch, indel_penalty=gap
def global_alignment( s: str, t: str, match_reward=match, mismatch_penalty=mismatch, indel_penalty=gap):
        backtrack, score = GlobalAlignmentBacktrack(s, t, indel_penalty, match_reward, mismatch_penalty)
        alignments = OutputGlobalAlignment(backtrack, t, s)
        return score[len(t)][len(s)], alignments[0], alignments[1]

In [4]:
# helper function: splits s and t after common subsequence alignment
def split_string(s, t, string):

    s_idx = s.find(string)
    t_idx = t.find(string)

    # Extract left and right parts
    s_left, s_right = s[:s_idx], s[s_idx + len(string):]
    t_left, t_right = t[:t_idx], t[t_idx + len(string):]

    return s_left, s_right, t_left, t_right

In [5]:
# helper function: find optimal common shared subsequence
# premise 
#   uni-alignments are more evolutionarily relevant
#   longer substrings tend to be evolutionarily conserved  

"""
    lcp_array(comb, suffix_a)
    generates the lcp array

    Args:
        comb (str): combination of both strings with "$" appended to the  (s + "$" + t + "#")
        suffix_a (array): suffix array 

        suffix array 
        - all the suffixes in sorted order with the index in which they occur 

    Returns:
        array: the lcp array 
"""

def lcp_array(comb, suffix_a):
    # constructs an lcp array
    rank = [0] * len(comb)
    lcp = [0] * len(comb)
    
    for i, suffix in enumerate(suffix_a):
        rank[suffix] = i
    
    idx = 0
    for i in range(len(comb)):
        if rank[i] > 0:
            j = suffix_a[rank[i] - 1]
            while (i + idx < len(comb)) and (j + idx < len(comb)) and (comb[i + idx] == comb[j + idx]):
                idx += 1
            lcp[rank[i]] = idx
            if idx > 0:
                idx -= 1
    return lcp


"""
    function:
    finding thing optimal common shared subsequence 
    only returns one substring 

    Args:
        s: string S 
        t: string T
        k1: int (such that k1 < k2 -- see tandem_transform_lcs)
    
    Returns: 
        longest: string 
        the longest common subsequence between strings S and T. 
        if there are two longest common subsequences of the same length, 
            return the one which maximizes the global alignment score.
        
        our code breaks if there are no matches between S and T 
        
"""
def lus(s, t, k1):
    # find longest unique substrings between s and t
    comb = s + "$" + t + "#"

    # craete suffix array
    suffixes = [(comb[i:], i) for i in range(len(comb))]
    suffixes.sort()
    suffix_array = [suffix[1] for suffix in suffixes]
    
    # create lcp array
    lcp = lcp_array(comb, suffix_array)
    
    # find all unique substrings
    all_shared_substrings = []

    for i in range(1, len(comb)):
        suff_one = suffix_array[i]
        suff_two = suffix_array[i - 1]

        if (suff_one < len(s) and suff_two > len(s)) or (suff_one > len(s) and suff_two < len(s)):
            if lcp[i] > 0:
                if (i-1) >= 0:
                     prev = lcp[i - 1] 
                else:
                     prev = 0
                if (i+1) < len(lcp):
                     next = lcp[i+1]
                else:
                     next = 0

                if lcp[i] > max(prev, next):
                    shared = comb[suff_one : suff_one + lcp[i]]
                    all_shared_substrings.append(shared)
    
    # find longest string with most optimal score
    max_len = 0
    for string in all_shared_substrings:
        max_len = max(len(string), max_len)

    longest = None
    cur_score = float("-inf")

    for string in all_shared_substrings:
        if len(string) == max_len and len(string) > k1:
            sl, sr, tl, tr = split_string(s, t, string)
            score = global_alignment(sl, tl)[0] + global_alignment(sr, tr)[0]
            if score > cur_score:
                longest = string
    return longest


In [6]:
# helper function: determine search space
# all kmers in left or right edge of alignment that can lead to tandem duplication / deletion
def get_search_space_right(s_sub, t_sub, k1, k2, common_alignment):

    temp = []
    for i in range(k1, k2+1):
        temp.append(common_alignment[len(common_alignment) - i:])

    search_space = []
    for kmer in temp:
        # tandem deletion
        kmer = kmer.replace(" ", "")

        if s_sub[:len(kmer)] == kmer and t_sub[:len(kmer)] != kmer:
            search_space.append((kmer, 'del'))

        # tandem duplication
        if s_sub[:len(kmer)] != kmer and t_sub[:len(kmer)] == kmer:
            search_space.append((kmer, 'dup'))

    return search_space

def get_search_space_left(s_sub, t_sub, k1, k2, common_alignment):

    temp = []
    for i in range(k1, k2+1):
        temp.append(common_alignment[:i])

    search_space = []
    for kmer in temp:
        # tandem deletion
        kmer = kmer.replace(" ", "")

        if s_sub[len(common_alignment) - len(kmer) - 1:] == kmer and t_sub[len(common_alignment) - len(kmer) - 1:] != kmer:
            search_space.append((kmer, 'del'))

        # tandem duplication
        if s_sub[len(common_alignment) - len(kmer) - 1:] != kmer and t_sub[len(common_alignment) - len(kmer) - 1:] == kmer:
            search_space.append((kmer, 'dup'))

    return search_space
        

In [ ]:
def tandem_transform_lcs(s: str, t: str, k1: int, k2: int, v: int):
        # keep track of kmers
        kmers = []
        max_iters = 100
        iters = 0

        # This while loop ensures that all potential tandem transformations around alignment are explored
        while iters < max_iters:

                score = global_alignment(s, t)
                if score[0] >= v:
                        if iters == 0:
                                print("Strings similar enough.")
                        return (score[0], score[1], score[2], kmers)

                # find best possible alignment between s and t longer than k1, uses greedy approach
                common_alignment = lus(s, t, k1)

                # return if no common alignments exist
                if not common_alignment:
                        return (score[0], score[1], score[2], kmers)
                
                indices = [(x.start(), x.end() - 1) for x in re.finditer(re.escape(common_alignment), s)][0]

        
                # align s and t to common prefix, get the left flanking and right flanking sequence
                sl, sr, tl, tr = split_string(s, t, common_alignment)

                # left side applied transformations

                # left kmer search space
                search_space_l = get_search_space_left(sl, tl, k1, k2, common_alignment)

                # test left side search space values and find the one that leads to highest global alignment score when applied
                max_score = float("-inf")
                transformation = ""
                for transforms in search_space_l:
                        if transforms[-1] == "del":
                                s_trans = sl[:len(sl) - len(transforms[0])]
                        else:
                                s_trans = sl + transforms[0]
                        reconstructed = s_trans + common_alignment  + s[indices[-1] + 1:]

                        # apply transformation and check alignment score
                        score = global_alignment(reconstructed, t)

                        if score[0] > v:
                                # if threshold is reached, a tandem transformation has been found
                                kmers.append(transforms)
                                return (score[0], score[1], score[2], kmers)
                        else:
                                # assign the highest scoring transformation 
                                if score[0] > max_score:
                                        transformation = reconstructed
                                        target_kmer = transforms

                # right kmer search space
                if search_space_l:
                        s = transformation
                        kmers.append(target_kmer)
                indices = [(x.start(), x.end() - 1) for x in re.finditer(re.escape(common_alignment), s)][0]
                search_space_r = get_search_space_right(sr, tr, k1, k2, common_alignment)


                # test right side search space values and select the one that leads to highest global alignment score when applied
                transformation_r = ""
                for transforms in search_space_r:

                        if transforms[-1] == "del":
                                s_trans = sr[len(transforms[0]):]
                        else:
                                s_trans = transforms[0] + sr
        
                        reconstructed = s[:indices[-1] + 1] + s_trans

                        # apply transformation and check alignment score
                        # score = similarity(reconstructed, t)
                        score = global_alignment(reconstructed, t)

                        if score[0] > v:
                                # if threshold is reached, a tandem transformation has been found
                                kmers.append(transforms)
                                return (score[0], score[1], score[2], kmers)
                        else:
                                # assign the highest scoring transformation 
                                if score[0] > max_score:
                                        transformation_r = reconstructed
                                        target_kmer = transforms

                # transformation on right side complete, final s transformation for this iteration
                if search_space_r:
                        s = transformation_r
                        kmers.append(target_kmer)

                # no more possible tandem duplications or deletions
                if not search_space_l and not search_space_r:
                        print("All tandem transformations exhausted for this alignment.")
                        return (score[0], score[1], score[2], kmers)
                
                iters += 1
                
        return 


### Testing Tandem Transformation

In [78]:
# basic test case to ensure proper alignment and detection of deletion
tandem_transform_lcs("TACCATCATC",
                     "ACCATCTT",
                     2,
                     3,
                     15)

All tandem transformations exhausted for this alignment.


(6, 'TACCATC--', '-ACCATCTT', [('ATC', 'del')])

In [ ]:
# This test case accounts for repeated tandem duplications
tandem_transform_lcs("AGACACACACACACACAC",
                     "CTTCTTAGACA",
                     2,
                     3,
                     3)

All tandem transformations exhausted for this alignment.


(-4,
 '------AGACAC',
 'CTTCTTAGACA-',
 [('CA', 'del'),
  ('CA', 'del'),
  ('CA', 'del'),
  ('CA', 'del'),
  ('CA', 'del'),
  ('CA', 'del')])

In [ ]:
# This test case accounts for same length string and similar alignment reached before all possible options exhausted
tandem_transform_lcs("CCTAAAACTCTA",
                     "CCTAACTCTCTA",
                     2,
                     3,
                     20)

(24, 'CCTAACTCTCTA', 'CCTAACTCTCTA', [('AA', 'del'), ('CT', 'dup')])